In [1]:
import json
import requests
from urllib.parse import urlencode, quote

In [2]:
class FortiAPIClient():
    
    def __init__(self, host, session=None, is_https=True, timeout=30):
        self._host = host
        self._session = session or requests.session()
        self._is_https = is_https
        self.timeout = timeout

    @property
    def host(self):
        return self._host
        
    @property
    def session(self):
        return self._session
        
    @property
    def is_https(self):
        return self._is_https
        
    @property
    def url_root(self):
        url_root = f'https://{self._host}' if self._is_https else f'http://{self._host}'
        return url_root
        
    def login(self, username, password, path='/logincheck'):
        url = self.url_root + path
        data = {
            'username': username,
            'secretkey': password,
        }
        encoded_data = urlencode(data)
        response = self._session.post(url=url, data=encoded_data)
        return response
    
    def logout(self, path='/logout'):
        url = self.url_root + path
        response = self._session.post(url=url)
        return response
    
    def get(self, path, params={}):
        url = self.url_root + path
        response = self._session.get(
            url=url,
            params=params,
            timeout=self.timeout,
        )
        return response
    
    def post(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.post(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response
    
    def put(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.put(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response
    
    def delete(self, path):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.delete(
            url=url,
            headers=headers,
            timeout=self.timeout,
        )
        return response

In [3]:
# Usage
# Login
client = FortiAPIClient('10.210.201.5', is_https=False, timeout=30)
client.login(username='admin', password='forti666')

<Response [200]>

In [4]:
# Get
response = client.get(
    path='/api/v2/cmdb/firewall/address',
    params={'format': 'name|subnet'}
)
print(response.text)

{
  "http_method":"GET",
  "revision":"697635a340a7185f07bb117aec459a03",
  "results":[
    {
      "name":"FABRIC_DEVICE",
      "q_origin_key":"FABRIC_DEVICE",
      "subnet":"0.0.0.0 0.0.0.0"
    },
    {
      "name":"FIREWALL_AUTH_PORTAL_ADDRESS",
      "q_origin_key":"FIREWALL_AUTH_PORTAL_ADDRESS",
      "subnet":"0.0.0.0 0.0.0.0"
    },
    {
      "name":"SSLVPN_TUNNEL_ADDR1",
      "q_origin_key":"SSLVPN_TUNNEL_ADDR1"
    },
    {
      "name":"address__10.210.201.168\/32",
      "q_origin_key":"address__10.210.201.168\/32",
      "subnet":"10.210.201.168 255.255.255.255"
    },
    {
      "name":"all",
      "q_origin_key":"all",
      "subnet":"0.0.0.0 0.0.0.0"
    },
    {
      "name":"gmail.com",
      "q_origin_key":"gmail.com"
    },
    {
      "name":"login.microsoft.com",
      "q_origin_key":"login.microsoft.com"
    },
    {
      "name":"login.microsoftonline.com",
      "q_origin_key":"login.microsoftonline.com"
    },
    {
      "name":"login.windows.net",
   

In [5]:
# Read & write
response.json()

{'http_method': 'GET',
 'revision': '697635a340a7185f07bb117aec459a03',
 'results': [{'name': 'FABRIC_DEVICE',
   'q_origin_key': 'FABRIC_DEVICE',
   'subnet': '0.0.0.0 0.0.0.0'},
  {'name': 'FIREWALL_AUTH_PORTAL_ADDRESS',
   'q_origin_key': 'FIREWALL_AUTH_PORTAL_ADDRESS',
   'subnet': '0.0.0.0 0.0.0.0'},
  {'name': 'SSLVPN_TUNNEL_ADDR1', 'q_origin_key': 'SSLVPN_TUNNEL_ADDR1'},
  {'name': 'address__10.210.201.168/32',
   'q_origin_key': 'address__10.210.201.168/32',
   'subnet': '10.210.201.168 255.255.255.255'},
  {'name': 'all', 'q_origin_key': 'all', 'subnet': '0.0.0.0 0.0.0.0'},
  {'name': 'gmail.com', 'q_origin_key': 'gmail.com'},
  {'name': 'login.microsoft.com', 'q_origin_key': 'login.microsoft.com'},
  {'name': 'login.microsoftonline.com',
   'q_origin_key': 'login.microsoftonline.com'},
  {'name': 'login.windows.net', 'q_origin_key': 'login.windows.net'},
  {'name': 'none',
   'q_origin_key': 'none',
   'subnet': '0.0.0.0 255.255.255.255'},
  {'name': 'wildcard.dropbox.com', '

In [6]:
# Post
response = client.post(
    path='/api/v2/cmdb/firewall/address',
    json={
        'name': 'address 10.210.201.168/32',
        'type': 'ipmask',
        'subnet': '10.210.201.168 255.255.255.255',
    }
)
print(response.text)

{
  "http_method":"POST",
  "revision":"37f26214418c317f3a12fe13cae2afba",
  "revision_changed":true,
  "old_revision":"697635a340a7185f07bb117aec459a03",
  "mkey":"address 10.210.201.168\/32",
  "status":"success",
  "http_status":200,
  "vdom":"root",
  "path":"firewall",
  "name":"address",
  "serial":"FGVMEVBH2GTGR773",
  "version":"v6.4.2",
  "build":1723
}


In [12]:
# Put
response = client.put(
    path='/api/v2/cmdb/firewall/address' + '/' + quote('address 10.210.201.168/32', safe=''),
    json={
        'name': 'address__10.210.201.168/32',
    }
)
print(response.json())

{'http_method': 'PUT', 'revision': '44b1cae150376bcd770dc70409aecae8', 'revision_changed': False, 'status': 'error', 'http_status': 404, 'vdom': 'root', 'path': 'firewall', 'name': 'address', 'mkey': 'address 10.210.201.168/32', 'serial': 'FGVMEVBH2GTGR773', 'version': 'v6.4.2', 'build': 1723}


In [11]:
# Delete
response = client.delete(
    path='/api/v2/cmdb/firewall/address' + '/' + quote('address 10.210.201.168/32', safe=''),
)
print(response.text)

{
  "http_method":"DELETE",
  "revision":"44b1cae150376bcd770dc70409aecae8",
  "revision_changed":false,
  "status":"error",
  "http_status":404,
  "vdom":"root",
  "path":"firewall",
  "name":"address",
  "mkey":"address 10.210.201.168\/32",
  "serial":"FGVMEVBH2GTGR773",
  "version":"v6.4.2",
  "build":1723
}
